In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('recognition_sample_submission.csv')
df.head()

,id,landmarks
0,cc4e1335111d0f14,8815 0.1
1,75f7113a26d670d9,8815 0.1
2,3eeb10b0aa3cfbff,8815 0.1
3,cefe5f969b9d4324,8815 0.1
4,9324ef326508dda3,8815 0.1


In [9]:
a = np.array([7, 7, 8, 9, 9])

In [10]:
unique_elements, counts_elements = np.unique(a, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))


Frequency of unique values of the said array:
[[7 8 9]
 [2 1 2]]
